In [16]:
from dotenv import load_dotenv
from openai import OpenAI
from pinecone import Pinecone

load_dotenv()
client = OpenAI() # api_key=

pc = Pinecone() # api_key=
index = pc.Index("ta")

Extract texts from the key frames.

In [8]:
import json
import os
import cv2
import os
import pytesseract
from PIL import Image
import datetime

pytesseract.pytesseract.tesseract_cmd = r'C:\Program Files\Tesseract-OCR\tesseract.exe'

def extract_text_from_image(image_path):
    with Image.open(image_path) as img:
        text = pytesseract.image_to_string(img)
        return text

def get_subdirectories(folder_path):
    # 获取指定文件夹下的所有文件和文件夹
    entries = os.listdir(folder_path)
    # 筛选出所有文件夹名
    subdirectories = [entry for entry in entries if os.path.isdir(os.path.join(folder_path, entry))]
    return subdirectories

frame_texts={}

root_path='./key_frames'
folders=get_subdirectories(root_path)
for folder in folders:
    image_texts={}
    folder_path=root_path+'/'+folder
    for image in os.listdir(folder_path):
        image_path=folder_path+'/'+image

        # Extract text from the frame
        text = extract_text_from_image(image_path)
        if text:
            time=':'.join(image[:-4].split('_')[1:])
            image_texts[time]=text
        
    frame_texts[folder]=image_texts

with open('key_frame_text.json', "w") as json_file:
    json.dump(frame_texts, json_file)      

Read the above file, and the YouTube video info file, combine them to a complete YouTube info file with frame time and texts.

In [11]:
import json
with open('./key_frame_text.json', "r") as json_file:
    video_frames = json.load(json_file)
with open('./YouTube_video_info.json', "r") as json_file:
    video_infos = json.load(json_file)

video_info_with_frames={}
for video_name, video_info in video_infos.items():
    name=video_name.replace("'",'')+'.mp4'
    if name not in video_frames:
        print(f'name is not a key in key_frame_text.json')
    video_info_with_frames[name]=video_info
    video_info_with_frames[name]['frames']=video_frames[name]
with open('YouTube_video_info_frames.json', "w") as json_file:
    json.dump(video_info_with_frames, json_file) 

Delete the unused video data in Pinecone, then do the embedding and upsert vectors into Pinecone.

In [ ]:
index.delete(delete_all=True, namespace='video_data')

In [18]:
import json
with open('./YouTube_video_info_frames.json', "r") as json_file:
    video_info_frames = json.load(json_file)

for file_name, info in video_info_frames.items():
    link=info['link']
    id=info['id']
    frames=info['frames']

    for time_s, frame_content in frames.items():
        res = client.embeddings.create(input=[frame_content], model="text-embedding-3-small")
        vector=res.data[0].embedding
        index.upsert(
            vectors=[
                {
                "id": f"Video-{file_name}-{time_s}", 
                "values": vector, 
                "metadata": {"video_id": id,"link": link, "file_name": file_name, "time": time_s}
                }
            ],
            namespace='video_data')